In [75]:
import pandas as pd
import re

In [76]:
df = (pd.read_json("UNHCR speeches/speeches.json"))
df = df[['author', 'content']]

In [77]:
## Extract titles from content - text up until the first date is mentioned.

df['Title'] = df['content'].str.extract(r"\n+([\s\S]+), \d+\s\w+\s\d+")

In [78]:
### Extract speeches and dates from content

## Speeches follow "Statements by the High Commissioner" and the date
date_text = (df['content'].
             str.extract(r"Statements by High Commissioner,\s*(\d+\s\w+\s\d+)[\s\n\r]+([\s\S]+)")
            )
date_text.columns = ['date', 'speech']

In [79]:
## Convert string dates to datetime
df['date'] = pd.to_datetime(date_text.date)

## Replace line breaks, double spaces with spaces
df['speech'] = date_text.speech.replace(to_replace = ['\n', '  '], value = ' ', regex = True)

## Remove dates
df['speech'] = df.speech.replace(to_replace = r"(\d+\s\w+\s\d+)", value = '', regex = True)

In [83]:
## Superficial capitalization switches for my own satisfaction

df['author'] = (df.author.replace
                (to_replace = ['ogata', 
                               'guterres', 
                               'lubbers', 
                               'khan', 
                               'hartling', 
                               'schnyder', 
                               'lindt', 
                               'hocké', 
                               'stoltenberg'],
                 value      = ['Ogata', 
                               'Guterres', 
                               'Lubbers', 
                               'Khan', 
                               'Hartling', 
                               'Schnyder', 
                               'Lindt', 
                               'Hocké', 
                               'Stoltenberg'])
               )

df = df.rename(columns={'Title' : 'title'})

In [85]:
## Final dataset
df = df[['author', 'date', 'title', 'speech']]
df.sample(10)

,author,date,title,speech
30,Guterres,2012-10-05,Forced displacement in 2012: Current dilemmas ...,This text is an edited version of the remarks ...
201,Ogata,2000-07-18,"""Protecting People on the Move"" - Address by M...","Ladies and Gentlemen, It is a pleasure to be ..."
180,Lubbers,2001-10-01,"Opening Statement by Mr. Ruud Lubbers, United ...","Mr. Chairman,Excellencies,Distinguished Delega..."
193,Ogata,2000-10-14,"Speech by Mrs. Sadako Ogata, United Nations Hi...","President Shim Yoon-Chong,Colleagues of Sungky..."
691,Goedhart,1952-09-15,"Speech by Dr. Gerrit Jan van Heuven Goedhart, ...","Mr. President, I have much pleasure in welcom..."
519,Hartling,1981-12-10,"Acceptance Speech by Mr. Poul Hartling, United...","Your Majesty,Your Royal Highnesses,Your Excell..."
14,Guterres,2013-09-30,High-Level Segment on Solidarity and Burden-Sh...,"Geneva, As delivered Madam Chairperson, Honor..."
265,Ogata,1998-04-27,"Statement of Mrs. Sadako Ogata, United Nations...","Ladies and Gentlemen, I would like to thank t..."
638,Schnyder,1962-05-14,"Statement by Mr. Felix Schnyder, United Nation...",The documents now before the Executive Committ...
284,Ogata,1997-10-20,"Statement by Mrs. Sadako Ogata, United Nations...",This is the first day of my one-week visit to ...


In [88]:
df.to_json(r'data/cleaned_speeches.json')